# Food Prices time-series dataset released by WFP

Link to Food Prices data released by WFP: https://dataviz.vam.wfp.org/economic_explorer/prices

In [1]:
from IPython.display import HTML as html

html('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [1]:
from IPython.display import display
from ipywidgets import widgets, interact
from interactive_map import interactive_map
from functools import partial
import plotly.graph_objects as go
import geopandas as gpd 
import seaborn as sns
import pandas as pd
import numpy as np
import ntpath
import glob

In [2]:
# Read the data released by WFP regarding the market prices indicator.
path = "./wfp_data/"
all_files = glob.glob(path + "*.csv")

dfs = []

for filename in all_files:
    df = pd.read_csv(filename)
    df["Country"] = ntpath.basename(filename).split(".")[0]
    dfs.append(df)

df = pd.concat(dfs, axis = 0, ignore_index = True)
print("The data released by wfp:")
df.head()

The data released by wfp:


,Country,Admin 1,Market,Commodity,Price Type,Year,Month,Price,Unit,Currency,Data Source
0,Burkina Faso,Boucle Du Mouhoun,Tougan,Maize,Retail,2003,1,156.2500,KG,XOF,SIM/SONAGESS
1,Burkina Faso,Boucle Du Mouhoun,Tougan,Maize,Retail,2003,2,156.2500,KG,XOF,SIM/SONAGESS
2,Burkina Faso,Boucle Du Mouhoun,Tougan,Maize,Retail,2003,3,153.5156,KG,XOF,SIM/SONAGESS
3,Burkina Faso,Boucle Du Mouhoun,Tougan,Maize,Retail,2003,4,156.2500,KG,XOF,SIM/SONAGESS
4,Burkina Faso,Boucle Du Mouhoun,Tougan,Maize,Retail,2003,5,156.2500,KG,XOF,SIM/SONAGESS


As you note, the data have a monthly temporal granularity. I adjust the datetime object and I rename the columns name.

In [3]:
year = 2018

In [4]:
print("Select only the data beyond the year %d." % year)
df = df.loc[df.Year >= year]
# Create an appropriate datetime object.
data = pd.to_datetime(df["Year"].astype(int).astype(str) + df["Month"].astype(int).astype(str), format = "%Y%m") 
df.insert(1, "Datetime", data)
df.drop(["Year", "Month"], axis = 1, inplace = True) # Drop duplicate infomation.
df.sort_values("Datetime", ascending = True, inplace = True) # Sort on datetime.
df.reset_index(drop = True, inplace = True)
# Rename the column of the adminstrata.
df.rename(columns = {"Admin 1": "AdminStrata"}, inplace = True)
df.head()

Select only the data beyond the year 2018.


,Country,Datetime,AdminStrata,Market,Commodity,Price Type,Price,Unit,Currency,Data Source
0,Burkina Faso,2018-01-01,Boucle Du Mouhoun,Tougan,Maize,Retail,181.0,KG,XOF,SIM/SONAGESS
1,Syria,2018-01-01,Homs,Qaryatein,Fuel (diesel),Retail,225.0,L,SYP,Field Monitors/Homs Sub Office
2,Nigeria,2018-01-01,Lagos,Lagos,Groundnuts (shelled),Wholesale,NaN,100 KG,NGN,NaN
3,Syria,2018-01-01,Idleb,Idleb,Eggs,Retail,1000.0,30 pcs,SYP,Field Monitors/Gazientap Sub Office
4,Syria,2018-01-01,As_Suweida,Shahba,Beans (white),Retail,550.0,KG,SYP,Field Monitors/Thrid Party Monitor (TPM)


### Brief items description

In [5]:
print("Check if the dataframe contains NaN values:")
df.isnull().sum()

Check if the dataframe contains NaN values:


Country           0
Datetime          0
AdminStrata       0
Market            0
Commodity         0
Price Type        0
Price          6784
Unit              0
Currency          0
Data Source    6784
dtype: int64

### Country item

In [6]:
print("The countries are:", ", ".join(df.Country.unique()))

The countries are: Burkina Faso, Syria, Nigeria, Yemen


### AdminStrata item

In [7]:
# Rename the name of some AdminStrata of Syria according to the fcs and rcsi names of the AdminStrata.
df.replace(["As_Suweida", "Dara", "Damascus", "City_Damascus", "Hassakeh", "Raqqa", "Dayr_Az_Zor", "Al_Qunaytirah"], 
           ["As-Sweida", "Dar'a", "Rural Damascus", "Damascus", "Al-Hasakeh", "Ar-Raqqa", "Deir-ez-Zor", "Quneitra"], inplace = True)

In [8]:
# Rename the name of some AdminStrata of Burkina Faso according to the fcs and rcsi names of the AdminStrata.
df.replace(["Boucle Du Mouhoun", "Sud-ouest", "Plateau Central", "Centre-est", "Centre-ouest", "Centre-nord", "Centre-sud", "Hauts-bassins"], 
           ["Boucle-Du-Mouhoun", "Sud-Ouest", "Plateau-Central", "Centre-Est", "Centre-Ouest", "Centre-Nord", "Centre-Sud", "Hauts-Bassins"], inplace = True)

In [9]:
AdminStratas = df.groupby("Country")["AdminStrata"].unique()

- **Yemen**: the administrative division of Yemen is divided into two main divisions (governorates and districts). There are 22 governorates, including the capital Sana'a (Amanat Al Asimah) and Socotra Archipelago. The Food Prices dataframe has the values of 21 governorates (Socotra not included).

In [10]:
print(AdminStratas["Yemen"].shape)
AdminStratas["Yemen"]

(21,)


array(['Shabwah', 'Abyan', 'Amran', 'Amanat Al Asimah', 'Ibb', 'Taizz',
       'Al Bayda', 'Hajjah', 'Al Mahwit', 'Raymah', 'Aden', 'Lahj',
       'Al Maharah', 'Marib', 'Hadramaut', "Al Dhale'e", 'Dhamar',
       "Sa'ada", 'Al Hudaydah', "Sana'a", 'Al Jawf'], dtype=object)

- **Nigeria**: Nigeria is divided into 36 states. It also includes *The Federal Capital Territory* that is not a state and it is under the direct control of the federal government. The AdminStrata items of the Food Price dataframe cover 13 states.

In [11]:
print(AdminStratas["Nigeria"].shape)
AdminStratas["Nigeria"]

(13,)


array(['Lagos', 'Borno', 'Zamfara', 'Adamawa', 'Kano', 'Kebbi', 'Oyo',
       'Gombe', 'Kaduna', 'Katsina', 'Jigawa', 'Yobe', 'Sokoto'],
      dtype=object)

- **Syria**: the governatores of the Syria country are 14. The Food Price dataframe has the values of all 14 governatores.

In [12]:
print(AdminStratas["Syria"].shape)
AdminStratas["Syria"]

(14,)


array(['Homs', 'Idleb', 'As-Sweida', "Dar'a", 'Damascus', 'Al-Hasakeh',
       'Aleppo', 'Lattakia', 'Rural Damascus', 'Hama', 'Ar-Raqqa',
       'Quneitra', 'Deir-ez-Zor', 'Tartous'], dtype=object)

- **Burkina Faso**: Burkina Faso is divided into 13 administrative regions. The Food Price dataframe has the values of all 13 regions.

In [13]:
print(AdminStratas["Burkina Faso"].shape)
AdminStratas["Burkina Faso"]

(13,)


array(['Boucle-Du-Mouhoun', 'Sahel', 'Sud-Ouest', 'Plateau-Central',
       'Nord', 'Est', 'Centre-Est', 'Centre-Ouest', 'Cascades',
       'Hauts-Bassins', 'Centre-Nord', 'Centre-Sud', 'Centre'],
      dtype=object)

## Other items

In [14]:
print("The various type of commodities:")
df.Commodity.unique()

The various type of commodities:


array(['Maize', 'Fuel (diesel)', 'Groundnuts (shelled)', 'Eggs',
       'Beans (white)', 'Yogurt', 'Wheat flour', 'Rice', 'Tea', 'Oil',
       'Bread (shop)', 'Yam', 'Fuel (petrol-gasoline)', 'Sorghum',
       'Spinach', 'Potatoes', 'Sorghum (white)',
       'Wage (non-qualified labour)', 'Lentils', 'Millet',
       'Bread (bakery)', 'Livestock (sheep two-year-old male) ', 'Wheat',
       'Apples', 'Sugar', 'Pumpkin', 'Parsley', 'Rice (imported)',
       'Exchange rate (unofficial)', 'Maize (white)',
       'Milling cost (wheat)', 'Fish (sardine canned) ', 'Cheese',
       'Cowpeas (white)', 'Oil (vegetable)', 'Peas (yellow split) ',
       'Fish (tuna canned) ', 'Water (drinking)', 'Dates', 'Tomatoes',
       'Maize (yellow)', 'Milk (cow fresh) ', 'Eggplants',
       'Beans (kidney red)', 'Bread', 'Bulgur', 'Fuel (gas)', 'Bananas',
       'Salt', 'Salt (iodised)', 'Oil (palm)', 'Carrots',
       'Cowpeas (brown)', 'Gari (yellow)', 'Gari (white)',
       'Meat (chicken plucked) ', 'Wag

In [15]:
print("The unit of measure of the commodoties:")
df.Unit.unique()

The unit of measure of the commodoties:


array(['KG', 'L', '100 KG', '30 pcs', 'MT', 'Unit', '1.3 KG', 'Packet',
       'Day', 'USD/LCU', 'Head', '50 KG', '125 G', '185 G', 'Cubic meter',
       '10 KG', '100 L', '18 KG'], dtype=object)

In [16]:
print("Check if some commodities are measured using different unit type:")
commodities_units = dict()

def unit_check(group):
    if len(group.Unit.unique()) > 1:
        commodities_units[group.Commodity.unique()[0]] = list(group.Unit.unique())

df.groupby(["Commodity"]).apply(unit_check)
print(commodities_units)

Check if some commodities are measured using different unit type:
{'Bulgur': ['KG', 'MT'], 'Cowpeas (brown)': ['KG', '100 KG'], 'Cowpeas (white)': ['KG', '100 KG'], 'Eggs': ['30 pcs', 'Unit'], 'Fuel (gas)': ['10 KG', '18 KG'], 'Gari (white)': ['100 KG', 'KG'], 'Gari (yellow)': ['100 KG', 'KG'], 'Groundnuts (shelled)': ['100 KG', 'KG'], 'Livestock (sheep two-year-old male) ': ['Unit', 'Head'], 'Maize (white)': ['100 KG', 'KG'], 'Maize (yellow)': ['100 KG', 'KG'], 'Millet': ['KG', '100 KG'], 'Oil (palm)': ['100 L', 'L'], 'Oil (vegetable)': ['L', 'KG'], 'Rice': ['MT', 'KG'], 'Rice (imported)': ['KG', '50 KG', '100 KG'], 'Rice (local)': ['100 KG', 'KG'], 'Rice (milled local) ': ['100 KG', '50 KG'], 'Sorghum (brown)': ['KG', '100 KG'], 'Sorghum (white)': ['100 KG', 'KG'], 'Wheat': ['KG', 'MT'], 'Yam': ['KG', '100 KG']}


In [17]:
# Convert all the commodities with double unit to have a single unit.
def adjust_unit(group):
    def adjusting(row):
        a = row.Unit
        b = row.Price
        c = row.Commodity
        # Select the common unit of measure.
        if a == "KG" and c == "Oil (vegetable)":
            a = "L"
            b = row.Price*0.92      
        elif row.Unit == "100 KG":
            a = "KG"
            b = row.Price/100
        elif row.Unit == "50 KG":
            a = "KG"
            b = row.Price/50
        elif row.Unit == "MT":
            a = "KG"
            b = row.Price/1000
        elif row.Unit == "100 L":
            a = "L"
            b = row.Price/100
        elif row.Unit == "30 pcs":
            a = "Unit"
            b = row.Price/30
        elif row.Unit == "18 KG":
            a = "KG"
            b = row.Price/18
        elif row.Unit == "10 KG":
            a = "KG"
            b = row.Price/10
        elif a == "Head" and c == "Livestock (sheep two-year-old male) ":
            a = "Unit"
            b = row.Price
            
        return pd.Series([a, b, c])    
   
    if len(group.Unit.unique()) > 1:
        group[["Unit", "Price", "Commodity"]] = group.apply(adjusting, axis = 1)

    return group    

In [18]:
print("Convert all the commodities with double unit to have a single unit:")
df = df.groupby(["Commodity"]).apply(adjust_unit)
df.head()

Convert all the commodities with double unit to have a single unit:


,Country,Datetime,AdminStrata,Market,Commodity,Price Type,Price,Unit,Currency,Data Source
0,Burkina Faso,2018-01-01,Boucle-Du-Mouhoun,Tougan,Maize,Retail,181.000000,KG,XOF,SIM/SONAGESS
1,Syria,2018-01-01,Homs,Qaryatein,Fuel (diesel),Retail,225.000000,L,SYP,Field Monitors/Homs Sub Office
2,Nigeria,2018-01-01,Lagos,Lagos,Groundnuts (shelled),Wholesale,NaN,KG,NGN,NaN
3,Syria,2018-01-01,Idleb,Idleb,Eggs,Retail,33.333333,Unit,SYP,Field Monitors/Gazientap Sub Office
4,Syria,2018-01-01,As-Sweida,Shahba,Beans (white),Retail,550.000000,KG,SYP,Field Monitors/Thrid Party Monitor (TPM)


In [19]:
print("Check if some commodities are measured using different Unit type:")
commodities_units = dict()

df.groupby(["Commodity"]).apply(unit_check)
print(commodities_units)

Check if some commodities are measured using different Unit type:
{}


In [20]:
print("Add as suffix to each Commodity corresponding to the value unit:")
def add_unit(row):
    a = row.Commodity
    b = row.Unit
    return pd.Series(["".join([a, " (", b, ")"]), b])

df[["Commodity", "Unit"]] = df.apply(add_unit, axis = 1)
df.head()

Add as suffix to each Commodity corresponding to the value unit:


,Country,Datetime,AdminStrata,Market,Commodity,Price Type,Price,Unit,Currency,Data Source
0,Burkina Faso,2018-01-01,Boucle-Du-Mouhoun,Tougan,Maize (KG),Retail,181.000000,KG,XOF,SIM/SONAGESS
1,Syria,2018-01-01,Homs,Qaryatein,Fuel (diesel) (L),Retail,225.000000,L,SYP,Field Monitors/Homs Sub Office
2,Nigeria,2018-01-01,Lagos,Lagos,Groundnuts (shelled) (KG),Wholesale,NaN,KG,NGN,NaN
3,Syria,2018-01-01,Idleb,Idleb,Eggs (Unit),Retail,33.333333,Unit,SYP,Field Monitors/Gazientap Sub Office
4,Syria,2018-01-01,As-Sweida,Shahba,Beans (white) (KG),Retail,550.000000,KG,SYP,Field Monitors/Thrid Party Monitor (TPM)


In [21]:
print("The price types:")
df["Price Type"].unique()

The price types:


array(['Retail', 'Wholesale'], dtype=object)

In [22]:
print("The currencies are:")
df.Currency.unique()

The currencies are:


array(['XOF', 'SYP', 'NGN', 'YER'], dtype=object)

In [23]:
print(df[df.Country == "Yemen"].Currency.unique())
print(df[df.Country == "Nigeria"].Currency.unique())
print(df[df.Country == "Syria"].Currency.unique())
print(df[df.Country == "Burkina Faso"].Currency.unique())

['YER']
['NGN']
['SYP']
['XOF']


In [24]:
# Delete the unutilized columns.
df.drop(labels = ["Market", "Price Type", "Unit", "Currency", "Data Source"], axis = 1, inplace = True)
df.head()

,Country,Datetime,AdminStrata,Commodity,Price
0,Burkina Faso,2018-01-01,Boucle-Du-Mouhoun,Maize (KG),181.000000
1,Syria,2018-01-01,Homs,Fuel (diesel) (L),225.000000
2,Nigeria,2018-01-01,Lagos,Groundnuts (shelled) (KG),NaN
3,Syria,2018-01-01,Idleb,Eggs (Unit),33.333333
4,Syria,2018-01-01,As-Sweida,Beans (white) (KG),550.000000


In [25]:
# Create appropriate time-series subgroups with an appropriate datetime object. 
df = df.groupby(["Country", "AdminStrata", "Commodity"]).apply(lambda group: group.set_index("Datetime").resample("M").mean()).reset_index()
df.reset_index(drop = True, inplace = True)
df.head()

,Country,AdminStrata,Commodity,Datetime,Price
0,Burkina Faso,Boucle-Du-Mouhoun,Beans (niebe) (KG),2018-01-31,365.5
1,Burkina Faso,Boucle-Du-Mouhoun,Beans (niebe) (KG),2018-02-28,374.5
2,Burkina Faso,Boucle-Du-Mouhoun,Beans (niebe) (KG),2018-03-31,382.0
3,Burkina Faso,Boucle-Du-Mouhoun,Beans (niebe) (KG),2018-04-30,385.5
4,Burkina Faso,Boucle-Du-Mouhoun,Beans (niebe) (KG),2018-05-31,394.5


In [26]:
# Create a dataframe with multindex column in order to have a summary dataframe of the time-series.
df = df.set_index(["Datetime", "Country", "AdminStrata", "Commodity"]).unstack(["Country", "AdminStrata", "Commodity"])
df.columns = df.columns.droplevel(0)
#df.rename(mapper = lambda x: f"Price_{x}", axis = "columns", level = 1, inplace = True)
df.columns.rename("AdminStrata", level = 1, inplace = True)
freq = "M"
df.index.freq = freq
df.head()

Country           Burkina Faso                                              \
AdminStrata  Boucle-Du-Mouhoun                                               
Commodity   Beans (niebe) (KG) Maize (KG) Millet (KG) Rice (imported) (KG)   
Datetime                                                                     
2018-01-31               365.5      168.0      197.50                400.0   
2018-02-28               374.5      170.5      215.00                400.0   
2018-03-31               382.0      170.0      217.75                400.0   
2018-04-30               385.5      172.0      219.50                400.0   
2018-05-31               394.5      178.0      225.25                400.0   

Country                                                               \
AdminStrata                                                 Cascades   
Commodity   Sorghum (KG) Sorghum (local) (100 KG) Beans (niebe) (KG)   
Datetime                                                               
2018-01-31         178.5                  16250.0              368.0   
2018-02-28         184.5                  17000.0              402.0   
2018-03-31         208.5                  17750.0              382.0   
2018-04-30         190.0                  18000.0              393.0   
2018-05-31         204.5                  18000.0              409.0   

Country                                          ...  \
AdminStrata                                      ...   
Commodity   Maize (KG) Millet (KG) Sorghum (KG)  ...   
Datetime                                         ...   
2018-01-31       132.0       250.0        181.0  ...   
2018-02-28       135.0       250.0        200.0  ...   
2018-03-31       151.0       256.0        200.0  ...   
2018-04-30       158.0       275.0        200.0  ...   
2018-05-31       158.0       275.0        213.0  ...   

Country                         Yemen                                     \
AdminStrata                     Taizz                                      
Commodity   Peas (yellow split)  (KG) Potatoes (KG) Rice (imported) (KG)   
Datetime                                                                   
2018-01-31                      512.5         300.0                700.0   
2018-02-28                      512.5         300.0                700.0   
2018-03-31                      537.5         312.5                700.0   
2018-04-30                      500.0         325.0                700.0   
2018-05-31                      500.0         262.5                700.0   

Country                                         \
AdminStrata                                      
Commodity   Salt (KG) Sugar (KG) Tomatoes (KG)   
Datetime                                         
2018-01-31      100.0      370.0         300.0   
2018-02-28      100.0      370.0         300.0   
2018-03-31      100.0      325.0         212.5   
2018-04-30      112.5      350.0         475.0   
2018-05-31      150.0      350.0         275.0   

Country                                                                      \
AdminStrata                                                                   
Commodity   Wage (non-qualified labour) (Day) Wage (qualified labour) (Day)   
Datetime                                                                      
2018-01-31                             3000.0                        6000.0   
2018-02-28                             3000.0                        6000.0   
2018-03-31                             3000.0                        6000.0   
2018-04-30                             3000.0                        6000.0   
2018-05-31                             3000.0                        6000.0   

Country                                  
AdminStrata                              
Commodity   Wheat (KG) Wheat flour (KG)  
Datetime                                 
2018-01-31       200.0            250.0  
2018-02-28       200.0            250.0  
2018-03-31       200.0            245.0  
2

In [27]:
df.to_csv("./time-series/wfp_market_price.csv", index_label = False)

## Example of food price for a selected country

In [28]:
df_all_country = df.copy()

In [29]:
def select_country(countries):
    global df, Country
    df = df_all_country[countries]
    Country = countries
    return df.head()

w = widgets.ToggleButtons(options = list(df_all_country.columns.levels[0]), description = "Country:", disabled = False)
p = interact(select_country, countries = w)

interactive(children=(ToggleButtons(description='Country:', options=('Burkina Faso', 'Nigeria', 'Syria', 'Yeme…

In [30]:
def plot(df, adminstrata):       
    # Time-series of the adminstrata.      
    group = df[adminstrata]

    # Adjust time-series group.
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]

    def subplot(graph):
        if graph == "Time-series":
            # Create figure.
            # Set default trace colors with colorway.
            colorway = sns.color_palette("hls", 8).as_hex()
            layout = go.Layout(colorway = colorway)

            fig = go.Figure(layout = layout)

            for col in group.columns:
                fig.add_trace(go.Scatter(x = group.index, y = group[col], name = col, mode = "lines+markers", line = dict(width = 1.5)))

            # Edit the layout.
            fig.update_layout(title = dict(text = adminstrata, y = 0.9, x = 0.5), 
                              yaxis_title = dict(text = "Price", font = dict(size = 10)))
            # Add legend title.
            fig.update_layout(annotations = [dict(x = 1.12, y = 1.1, align = "right", valign = "top", text = "Indicator", 
                                                  showarrow = False, xref = "paper", yref = "paper", xanchor = "center", 
                                                  yanchor = "top")])
            # Add range slider.
            fig.update_layout(xaxis = dict(title = "Datetime", rangeselector = dict(), rangeslider = dict(visible = True), 
                                           type = "date"))

            fig.show()

        if graph == "Missing values":
            # Visualization of the missing values of the current time-series.
            miss = group.notnull().astype("int")
            fig1 = go.Figure(data = go.Heatmap(z = miss, zmin = 0, zmax = 1, x = miss.columns, y = miss.index, xgap = 3, 
                                               colorscale = [[0, "white"], [1, "black"]], showscale = False, 
                                               hoverinfo = "x+y"))
            # Edit the layout.
            fig1.update_layout(title = dict(text = adminstrata, y = 0.9, x = 0.5), coloraxis_showscale = False)
            fig1.update_layout(xaxis_title = "Indicator", yaxis_title = "Datetime")
            fig1.show()

    w = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)  
    p = interact(subplot, graph = w)

In [31]:
map1, out1 = interactive_map(Country, partial(plot, df))
display(map1)
display(out1)

Map(center=[9.3, 8], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

Output()

### Adjusting the time-series

In [32]:
# Linear interpolation of the dataframe.
df_interpolate = df.interpolate(method = "linear", limit = 4)
# Delete the columns that contains Nan values.
df_interpolate.dropna(inplace = True, axis = 1)
df_interpolate.head()

AdminStrata        Borno                                            \
Commodity   Bread (Unit) Cowpeas (brown) (KG) Cowpeas (white) (KG)   
Datetime                                                             
2018-01-31     352.00000            325.86155            274.45095   
2018-02-28     359.76745            331.85000            296.75000   
2018-03-31     391.20155            379.36250            308.80000   
2018-04-30     387.86825            395.50000            313.52500   
2018-05-31     400.13955            393.00000            314.50000   

AdminStrata                                                                 \
Commodity   Fuel (diesel) (L) Fuel (petrol-gasoline) (L) Gari (white) (KG)   
Datetime                                                                     
2018-01-31             224.70                    207.240        231.511233   
2018-02-28             228.25                    180.775        132.580000   
2018-03-31             229.50                    158.800        115.730000   
2018-04-30             231.25                    152.750        115.170000   
2018-05-31             239.90                    145.380        127.370000   

AdminStrata                                                                  \
Commodity   Gari (yellow) (KG) Groundnuts (shelled) (KG) Maize (white) (KG)   
Datetime                                                                      
2018-01-31           318.64065                 279.32925          95.384667   
2018-02-28           154.02000                 250.50000          92.900000   
2018-03-31           130.91250                 263.70000          93.500000   
2018-04-30           138.09000                 258.40000         100.000000   
2018-05-31           155.80000                 268.88000         102.200000   

AdminStrata                      ...            Zamfara                      \
Commodity   Maize (yellow) (KG)  ... Maize (white) (KG) Maize (yellow) (KG)   
Datetime                         ...                                          
2018-01-31              101.338  ...             94.403              94.544   
2018-02-28              101.250  ...            100.200              98.400   
2018-03-31              101.650  ...            104.150             104.450   
2018-04-30              104.500  ...            115.850             116.800   
2018-05-31              110.980  ...            118.040             114.340   

AdminStrata                                                                    \
Commodity   Millet (KG) Oil (palm) (L) Rice (imported) (KG) Rice (local) (KG)   
Datetime                                                                        
2018-01-31      123.338          406.4             294.6195          244.8227   
2018-02-28      122.400          439.3             283.6000          262.0000   
2018-03-31      131.550          420.9             278.1000          259.5500   
2018-04-30      135.800          454.4             276.0000          270.8000   
2018-05-31      138.680          456.8             276.8800          271.6800   

AdminStrata                                                 \
Commodity   Rice (milled local)  (KG) Sorghum (brown) (KG)   
Datetime                                                     
2018-01-31                     501.44              98.5714   
2018-02-28                     524.00              97.6000   
2018-03-31                     519.10             100.7000   
2018-04-30                     541.60             113.2000   
2018-05-31                     543.36             116.2800   

AdminStrata                                 
Commodity   Sorghum (white) (KG)  Yam (KG)  
Datetime                                    
2018-01-31             109.38585  148.7179  
2018-02-28             122.80000  318.0000  
2018-03-31             122.90000  141.8605  
2018-04-30             130.00000  150.0000  
2018-05-31             128.72000  169.5349  

[5 rows x 185 columns]

In [33]:
def plot(df, adminstrata):       
    # Time-series of the adminstrata.      
    group = df[adminstrata]
        
    # Adjust time-series group.
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]

    def subplot(graph):
        if graph == "Time-series":
            # Create figure.
            # Set default trace colors with colorway.
            colorway = sns.color_palette("hls", 8).as_hex()
            layout = go.Layout(colorway = colorway)

            fig = go.Figure(layout = layout)

            for col in group.columns:
                fig.add_trace(go.Scatter(x = group.index, y = group[col], name = col, mode = "lines+markers", line = dict(width = 1.5)))

            # Edit the layout.
            fig.update_layout(title = dict(text = adminstrata, y = 0.9, x = 0.5), 
                              yaxis_title = dict(text = "Price", font = dict(size = 10)))
            # Add legend title.
            fig.update_layout(annotations = [dict(x = 1.12, y = 1.1, align = "right", valign = "top", text = "Indicator", 
                                                  showarrow = False, xref = "paper", yref = "paper", xanchor = "center", 
                                                  yanchor = "top")])
            # Add range slider.
            fig.update_layout(xaxis = dict(title = "Datetime", rangeselector = dict(), rangeslider = dict(visible = True), 
                                           type = "date"))

            fig.show()

        if graph == "Missing values":
            # Visualization of the missing values of the current time-series.
            miss = group.notnull().astype("int")
            fig1 = go.Figure(data = go.Heatmap(z = miss, zmin = 0, zmax = 1, x = miss.columns, y = miss.index, xgap = 3, 
                                               colorscale = [[0, "white"], [1, "black"]], showscale = False, 
                                               hoverinfo = "x+y"))
            # Edit the layout.
            fig1.update_layout(title = dict(text = adminstrata, y = 0.9, x = 0.5), coloraxis_showscale = False)
            fig1.update_layout(xaxis_title = "Indicator", yaxis_title = "Datetime")
            fig1.show()

    w = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)  
    p = interact(subplot, graph = w)

In [34]:
map2, out2 = interactive_map(Country, partial(plot, df_interpolate))
display(map2)
display(out2)

Map(center=[9.3, 8], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

Output()

### Fit of the time-series

In [35]:
# I get the dataframe with time step of the day making an interpolation.
df_fit = df_interpolate.resample("D").interpolate(method = "polynomial", order = 2)
freq = "D"
df_fit.index.freq = freq
df_fit.head()

AdminStrata        Borno                                            \
Commodity   Bread (Unit) Cowpeas (brown) (KG) Cowpeas (white) (KG)   
Datetime                                                             
2018-01-31    352.000000           325.861550           274.450950   
2018-02-01    351.752463           325.278044           275.448932   
2018-02-02    351.543811           324.753603           276.431981   
2018-02-03    351.374043           324.288227           277.400098   
2018-02-04    351.243161           323.881916           278.353282   

AdminStrata                                                                 \
Commodity   Fuel (diesel) (L) Fuel (petrol-gasoline) (L) Gari (white) (KG)   
Datetime                                                                     
2018-01-31         224.700000                 207.240000        231.511233   
2018-02-01         224.871405                 206.215950        226.398647   
2018-02-02         225.039504                 205.197742        221.403048   
2018-02-03         225.204299                 204.185377        216.524436   
2018-02-04         225.365788                 203.178853        211.762811   

AdminStrata                                                                  \
Commodity   Gari (yellow) (KG) Groundnuts (shelled) (KG) Maize (white) (KG)   
Datetime                                                                      
2018-01-31          318.640650                279.329250          95.384667   
2018-02-01          310.068997                277.446412          95.260312   
2018-02-02          301.696776                275.626775          95.138595   
2018-02-03          293.523989                273.870340          95.019517   
2018-02-04          285.550634                272.177107          94.903077   

AdminStrata                      ...            Zamfara                      \
Commodity   Maize (yellow) (KG)  ... Maize (white) (KG) Maize (yellow) (KG)   
Datetime                         ...                                          
2018-01-31           101.338000  ...          94.403000           94.544000   
2018-02-01           101.331400  ...          94.682391           94.678967   
2018-02-02           101.325056  ...          94.956421           94.814137   
2018-02-03           101.318968  ...          95.225093           94.949511   
2018-02-04           101.313136  ...          95.488404           95.085089   

AdminStrata                                                                    \
Commodity   Millet (KG) Oil (palm) (L) Rice (imported) (KG) Rice (local) (KG)   
Datetime                                                                        
2018-01-31   123.338000     406.400000           294.619500        244.822700   
2018-02-01   123.112876     408.706851           294.117827        245.858547   
2018-02-02   122.901947     410.929862           293.624164        246.863107   
2018-02-03   122.705212     413.069031           293.138509        247.836380   
2018-02-04   122.522671     415.124360           292.660862        248.778367   

AdminStrata                                                 \
Commodity   Rice (milled local)  (KG) Sorghum (brown) (KG)   
Datetime                                                     
2018-01-31                 501.440000            98.571400   
2018-02-01                 502.863145            98.498298   
2018-02-02                 504.240554            98.428041   
2018-02-03                 505.572227            98.360629   
2018-02-04                 506.858165            98.296062   

AdminStrata                                   
Commodity   Sorghum (white) (KG)    Yam (KG)  
Datetime                                      
2018-01-31            109.385850  148.717900  
2018-02-01            110.148380  161.674388  
2018-02-02            110.889913  174.118972  
2018-02-03            111.610450  186.051652  
2018-02-04            112.309990  197.472429  

[5 rows x 185 columns]

In [36]:
def plot(df, adminstrata):       
    # Time-series of the adminstrata.      
    group = df[adminstrata]

    # Adjust time-series group.
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]

    def subplot(graph):
        if graph == "Time-series":
            # Create figure.
            # Set default trace colors with colorway.
            colorway = sns.color_palette("hls", 8).as_hex()
            layout = go.Layout(colorway = colorway)

            fig = go.Figure(layout = layout)

            for col in group.columns:
                fig.add_trace(go.Scatter(x = group.index, y = group[col], name = col, mode = "lines", line = dict(width = 1.5)))

            # Edit the layout.
            fig.update_layout(title = dict(text = adminstrata, y = 0.9, x = 0.5), 
                              yaxis_title = dict(text = "Price", font = dict(size = 10)))
            # Add legend title.
            fig.update_layout(annotations = [dict(x = 1.12, y = 1.1, align = "right", valign = "top", text = "Indicator", 
                                                  showarrow = False, xref = "paper", yref = "paper", xanchor = "center", 
                                                  yanchor = "top")])
            # Add range slider.
            fig.update_layout(xaxis = dict(title = "Datetime", rangeselector = dict(), rangeslider = dict(visible = True), 
                                           type = "date"))

            fig.show()

        if graph == "Missing values":
            # Visualization of the missing values of the current time-series.
            miss = group.notnull().astype("int")
            fig1 = go.Figure(data = go.Heatmap(z = miss, zmin = 0, zmax = 1, x = miss.columns, y = miss.index, xgap = 3, 
                                               colorscale = [[0, "white"], [1, "black"]], showscale = False, 
                                               hoverinfo = "x+y"))
            # Edit the layout.
            fig1.update_layout(title = dict(text = adminstrata, y = 0.9, x = 0.5), coloraxis_showscale = False)
            fig1.update_layout(xaxis_title = "Indicator", yaxis_title = "Datetime")
            fig1.show()

    w = widgets.RadioButtons(options = ["Time-series", "Missing values"], description = " ", disabled = False)  
    p = interact(subplot, graph = w)

In [37]:
map3, out3 = interactive_map(Country, partial(plot, df_fit))
display(map3)
display(out3)

Map(center=[9.3, 8], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

Output()

In [38]:
df_fit.to_csv("./time-series/wfp_market_price_fit-" + Country + ".csv", index_label = False)

### Common fitted time-series

In [39]:
# Keep only the commodities that are commons to each adminstrata.
commodities = list()
def common_commodities(group):
    group.columns = group.columns.droplevel()
    commodities.append(list(group.columns))

df_fit.groupby(axis = 1, level = 0).apply(common_commodities)
common = list(set.intersection(*map(set, commodities)))

In [40]:
df_common = df_fit.loc[:, pd.IndexSlice[:, common]]

In [41]:
df_common.head()

AdminStrata        Borno                                         \
Commodity   Bread (Unit) Cowpeas (white) (KG) Fuel (diesel) (L)   
Datetime                                                          
2018-01-31    352.000000           274.450950        224.700000   
2018-02-01    351.752463           275.448932        224.871405   
2018-02-02    351.543811           276.431981        225.039504   
2018-02-03    351.374043           277.400098        225.204299   
2018-02-04    351.243161           278.353282        225.365788   

AdminStrata                                               \
Commodity   Fuel (petrol-gasoline) (L) Gari (white) (KG)   
Datetime                                                   
2018-01-31                  207.240000        231.511233   
2018-02-01                  206.215950        226.398647   
2018-02-02                  205.197742        221.403048   
2018-02-03                  204.185377        216.524436   
2018-02-04                  203.178853        211.762811   

AdminStrata                                                                   \
Commodity   Groundnuts (shelled) (KG) Maize (white) (KG) Maize (yellow) (KG)   
Datetime                                                                       
2018-01-31                 279.329250          95.384667          101.338000   
2018-02-01                 277.446412          95.260312          101.331400   
2018-02-02                 275.626775          95.138595          101.325056   
2018-02-03                 273.870340          95.019517          101.318968   
2018-02-04                 272.177107          94.903077          101.313136   

AdminStrata                             ...           Zamfara  \
Commodity   Millet (KG) Oil (palm) (L)  ... Fuel (diesel) (L)   
Datetime                                ...                     
2018-01-31   120.876267     202.809334  ...        194.800000   
2018-02-01   121.388626     201.896752  ...        195.647265   
2018-02-02   121.887962     200.994368  ...        196.477273   
2018-02-03   122.374277     200.102184  ...        197.290023   
2018-02-04   122.847570     199.220197  ...        198.085515   

AdminStrata                                               \
Commodity   Fuel (petrol-gasoline) (L) Gari (white) (KG)   
Datetime                                                   
2018-01-31                  197.400000        135.456500   
2018-02-01                  199.775628        133.180598   
2018-02-02                  202.027135        130.976307   
2018-02-03                  204.154522        128.843626   
2018-02-04                  206.157788        126.782556   

AdminStrata                                                                   \
Commodity   Groundnuts (shelled) (KG) Maize (white) (KG) Maize (yellow) (KG)   
Datetime                                                                       
2018-01-31                 209.120000          94.403000           94.544000   
2018-02-01                 211.531416          94.682391           94.678967   
2018-02-02                 213.879764          94.956421           94.814137   
2018-02-03                 216.165045          95.225093           94.949511   
2018-02-04                 218.387257          95.488404           95.085089   

AdminStrata                                                  \
Commodity   Millet (KG) Oil (palm) (L) Sorghum (brown) (KG)   
Datetime                                                      
2018-01-31   123.338000     406.400000            98.571400   
2018-02-01   123.112876     408.706851            98.498298   
2018-02-02   122.901947     410.929862            98.428041   
2018-02-03   122.705212     413.069031            98.360629   
2018-02-04   122.522671     415.124360            98.296062   

AdminStrata                       
Commodity   Sorghum (white) (KG)  
Datetime                          
2018-01-31            109.385850  
2018-02-01            110.148380  
2018-02-02        

In [42]:
df_common.to_csv("./time-series/wfp_common_market_price-" + Country + ".csv", index_label = False)